## Acerca del conjunto de datos

El analisis de personalidad de clientes es un analisis detallado de los clientes ideales de una empresa. Ayuda a la empresa a conocer mejor a sus clientes y les permite modificar los productos de acuerdo con las necesidades, comportmaientos e inquietudes de los diferentes tipos de clientes.

Ayuda a la empresa a modificar su producto en funcion de su cliente objetivo de distintos tipos de segmentos de clientes. Ejemplo, en lugar de gastar dinero para comercializar un nuevo producto a todos los clientes, una empresa puede analizar que segmento de clientes tienen mas probabilidades de comprar el producto y luego comercializarlo solo en ese segmento en particular

### Cargar librerias

### Cargar Datos